In [1]:
%run useful_functions.py

C:\Users\natha\Desktop\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\Work_Nathan\useful_functions.py:951: SyntaxWarning: invalid escape sequence '\s'
  item1A = re.sub('\s{2,}',' ', item1A)
C:\Users\natha\Desktop\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\17-Cyber-risk_and_the_cross-section_of_stock_returns-main\Work_Nathan\useful_functions.py:962: SyntaxWarning: invalid escape sequence '\s'
  document = re.sub('\s{2,}',' ', document)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natha\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


update 21


# Linear regression

In [58]:
# Function to read text from a file
def read_text_from_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

# Function to extract Panel OLS summary information
def extract_summary_data(text):
    summary_regex = r"(?P<statistic>[a-zA-Z ()]+):\s+(?P<value>[-+e.\d]+)"
    summary_matches = re.findall(summary_regex, text)
    summary_data = {stat: float(value) for stat, value in summary_matches}
    return summary_data

# Function to extract Parameter Estimates information
def extract_parameters_data(text):
    parameters_regex = r"(?P<parameter>[a-zA-Z0-9_()[\] .-]+)\s+(?P<coefficient>[-+.\de]+)\s+(?P<stderr>[-+.\de]+)\s+(?P<tstat>[-+.\de]+)\s+(?P<pvalue>[-+.\de]+)\s+(?P<lower_ci>[-+.\de]+)\s+(?P<upper_ci>[-+.\de]+)"
    parameters_matches = re.findall(parameters_regex, text)
    parameters_data = {
        "Parameter": [match[0] for match in parameters_matches],
        "Coefficient": [float(match[1]) for match in parameters_matches],
        "Std. Err.": [float(match[2]) for match in parameters_matches],
        "T-stat": [float(match[3]) for match in parameters_matches],
        "P-value": [float(match[4]) for match in parameters_matches],
        "Lower CI": [float(match[5]) for match in parameters_matches],
        "Upper CI": [float(match[6]) for match in parameters_matches],
    }
    return parameters_data


def transform_p_value(p):
    if p<0.01:
        return "***"
    if p<0.05:
        return "**"
    if p<0.1:
        return "*"

    return ""


def get_lin_tables(file_path):


    # Read the text from the file
    text = read_text_from_file(file_path)
    
    # Extract data
    summary_data = extract_summary_data(text)
    parameters_data = extract_parameters_data(text)
    
    # Create dataframes
    summary_df = pd.DataFrame(summary_data.items(), columns=["Statistic", "Value"])
    parameters_df = pd.DataFrame(parameters_data)
    

    #re-format
    parameters_df = parameters_df[~parameters_df['Parameter'].str.contains("20")]
    parameters_df = parameters_df[~parameters_df['Parameter'].str.contains("T.")]
    parameters_df.drop(columns=["Std. Err.","Lower CI","Upper CI"],inplace=True)

    parameters_df['P-value'] = parameters_df['P-value'].apply(transform_p_value)

    parameters_df["T-stat"]=parameters_df["T-stat"].round(2)

    parameters_df["Coefficient"]=parameters_df["Coefficient"].round(4)


    parameters_df=parameters_df.astype(str)

    parameters_df["Coefficient"]="\textbf{"+parameters_df["Coefficient"]+parameters_df["P-value"]+"}"

    parameters_df.drop(columns=['P-value'],inplace=True)

    parameters_df["T-stat"]="["+parameters_df["T-stat"]+"]"

    parameters_df["Parameter"]=parameters_df["Parameter"].apply(lambda x : x.replace(" ",""))

    parameters_df.set_index("Parameter", inplace=True)

    parameters_df.index.name = None

    parameters_df = parameters_df.rename(index={
    "Intercept":"Constant",
    "firm_size":"Firm Size (ln)",
    "firm_age":"Firm Age (ln)",
    "book_to_market":"Book to Market",
    "TobinQ":"Tobin’s Q",
    "MktBeta":"Market Beta",
    "intangibles_to_assets":"Intangible/Assets",
    "debt_to_assets":"Debt/Assets",
    "price_to_earnings":"Price/Earnings",
    "profit_margin":"Profit Margin",
    "asset_turnover":"Asset Turnover",
    "cash_ratio":"Cash Ratio",
    "sales_to_invested_cap":"Sales/Invested Cap.",
    "capital_ratio":"Capital Ratio",
    "RD_to_sales":"R&D/Sales ",
    "readability":"Readability",
    "secret":"Secret",
    "risk_length_table":"Risk Length Table (ln)",
    "volume_per_cap":"Volume per Cap.",
    "humans_per_cap":"Humans per Cap."
    })

   
    summary_df.set_index("Statistic", inplace=True)
    return summary_df,parameters_df

In [75]:
# Define the path to your text file
pth='results_pipeline/super/Preparation_and_Reconaissance'
file_path1 = pth+'/PanelOLS_firm_year_fixed_effect.txt'
file_path2= pth+'/PanelOLS_industry_year_fixed_effect.txt'

summary_df1,parameters_df1=get_lin_tables(file_path1)
summary_df2,parameters_df2=get_lin_tables(file_path2)

In [76]:
parameters_df1.loc['Year fixed effect']=["Yes",""]
parameters_df1.loc['Industry fixed effect']=["No",""]
parameters_df1.loc['Firm fixed effect']=["Yes",""]
parameters_df1.loc['Observations']=[int(summary_df1["Value"].iloc[2]),""]
parameters_df1.loc['R2']=[summary_df1["Value"].iloc[0],""]


parameters_df2.loc['Year fixed effect']=["Yes",""]
parameters_df2.loc['Industry fixed effect']=["Yes",""]
parameters_df2.loc['Firm fixed effect']=["No",""]
parameters_df2.loc['Observations']=[int(summary_df2["Value"].iloc[2]),""]
parameters_df2.loc['R2']=[summary_df2["Value"].iloc[0],""]
                                         

In [77]:
display(parameters_df1)
display(parameters_df2)


,Coefficient,T-stat
Constant,\textbf{48.897***},[42.14]
Firm Size (ln),\textbf{0.0069},[0.13]
Firm Age (ln),\textbf{-0.7352***},[-5.92]
ROA,\textbf{-0.0807},[-0.76]
Book to Market,\textbf{-0.0347***},[-2.75]
Market Beta,\textbf{-0.0577*},[-1.92]
Intangible/Assets,\textbf{-0.1576},[-0.73]
Debt/Assets,\textbf{-0.4222*},[-1.67]
ROE,\textbf{0.0376},[0.7]
Price/Earnings,\textbf{0.0002},[1.6]


,Coefficient,T-stat
Constant,\textbf{49.466***},[56.36]
Firm Size (ln),\textbf{0.0873***},[2.99]
Firm Age (ln),\textbf{-0.523***},[-9.11]
ROA,\textbf{-0.1396},[-0.78]
Book to Market,\textbf{-0.1499***},[-5.58]
Market Beta,\textbf{-0.0962**},[-2.34]
Intangible/Assets,\textbf{1.5511***},[7.31]
Debt/Assets,\textbf{1.6297***},[3.81]
ROE,\textbf{0.0192},[0.21]
Price/Earnings,\textbf{0.0},[0.05]


# Univariate sort

In [78]:
path="results_pipeline/super/Preparation_and_Reconaissance/fama_french_factor_table.csv"

In [63]:
df = pd.read_csv(path)
df.rename(columns={'Unnamed: 0': ' '}, inplace=True)
df.set_index(' ', inplace=True)
df=df.round(2)

In [64]:
df.loc['t-stat'] = df.loc['t-stat'].astype(str).map(lambda x: '['+str(x)+']')

C:\Users\natha\AppData\Local\Temp\ipykernel_10868\3142652444.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['[3.54]' '[-0.44]' '[-0.14]' '[-0.95]']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc['t-stat'] = df.loc['t-stat'].astype(str).map(lambda x: '['+str(x)+']')
C:\Users\natha\AppData\Local\Temp\ipykernel_10868\3142652444.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['[3.13]' '[-1.97]' '[-2.45]' '[-3.08]']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc['t-stat'] = df.loc['t-stat'].astype(str).map(lambda x: '['+str(x)+']')
C:\Users\natha\AppData\Local\Temp\ipykernel_10868\3142652444.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of panda

In [65]:

new_index = "A. Portfolios sorted by cyber score"
new_row_content = [" "] * df.shape[1]  # Fill with spaces

# Insert the new row
df.loc[new_index] = new_row_content

In [66]:
df=df.rename(index={'t-stat': ' '})

In [67]:
# Create a new DataFrame for the new row
new_row_df = pd.DataFrame([new_row_content], columns=df.columns, index=[new_index])

# Concatenate the new row DataFrame with the original DataFrame
df = pd.concat([new_row_df, df])
df=df[:-1]

In [68]:
# New index and row content
new_index = "B. Characteristics"
new_row_content = [" "] * df.shape[1]  # Fill with spaces

# Calculate the index position to insert the new row
insert_index = len(df) - 3

# Create a new DataFrame for the new row
new_row_df = pd.DataFrame([new_row_content], columns=df.columns, index=[new_index])

# Concatenate the new row DataFrame with the original DataFrame
df = pd.concat([df.loc[:df.index[insert_index]], new_row_df, df.loc[df.index[insert_index]:]])



In [69]:
df.replace(-99, "-", inplace=True)

In [70]:
df=df.iloc[[0,1,2,3,4,5,6,7,8,10,11,12,13]]

In [71]:
df=df.rename(index={'nb firms': 'Nb. firms ',"avg cyber risk":"Avg. cyber score"})

In [72]:
def add_significance_markers(df):
    # Iterate over each row in the DataFrame starting from the second row
    for i in range(1, len(df)):
        # Iterate over each column
        for col in range(df.shape[1]):
            value = df.iloc[i, col]
            # Check if the value contains a square bracket
            if "[" in value and "]" in value:
                # Extract the t-statistic from the square brackets
                t_statistic = float(value.split("[")[1].split("]")[0])
                
                if t_statistic >= 2.58:
                    df.iloc[i - 1, col] += "***"
                elif t_statistic >= 1.96:
                    df.iloc[i - 1, col] += "**"
                elif t_statistic >= 1.65:
                    df.iloc[i - 1, col] += "*"

    return df

# Add significance markers
df_with_markers = add_significance_markers(df.astype(str))

# Print the DataFrame with significance markers
df_with_markers

,P1,P2,P3,P4,P5,P5-P1
A. Portfolios sorted by cyber score,,,,,,
avg. excess ret.,0.86***,0.85***,1.15***,1.11***,1.43***,0.57**
,[3.54],[3.13],[4.22],[3.94],[4.69],[1.97]
CAPM alpha,-0.09,-0.23,0.02,0.02,0.37**,0.46
,[-0.44],[-1.97],[0.2],[0.29],[2.39],[1.34]
FFC alpha,-0.02,-0.16,0.07,0.04,0.28***,0.3*
,[-0.14],[-2.45],[1.23],[0.56],[3.07],[1.75]
FF5 alpha,-0.09,-0.19,0.11*,0.01,0.3***,0.38**
,[-0.95],[-3.08],[1.76],[0.19],[3.17],[2.42]
B. Characteristics,,,,,,


In [73]:
#path="results_pipeline/Overall/model_"+str(i)+"_FamaMcBeth_regression.csv"

# Double sort

In [90]:
def show_double_sort_df(path):
    df = pd.read_csv(path)
    #df.rename(columns={'Unnamed: 0': ' '}, inplace=True)
    #df.set_index(' ', inplace=True)
    df=df.round(2)
    display(df)
    return df

In [114]:
path="results_pipeline/super/Preparation_and_Reconaissance"

In [115]:
a1=show_double_sort_df(path+"/avg_excess_returns_double_sort_market.csv")
a2=show_double_sort_df(path+"/avg_excess_returns_double_sort_book.csv")
a3=show_double_sort_df(path+"/avg_excess_returns_double_sort_size.csv")

,Unnamed: 0,Cyber Q1,Cyber Q2,Cyber Q3,Cyber Q4,Cyber Q5
0,Beta Q1,1.10,0.91,1.28,1.21,1.41
1,Beta Q2,0.93,0.87,1.20,1.13,1.35
2,Beta Q3,0.92,0.90,1.13,1.11,1.29
3,Beta Q4,0.90,0.77,1.20,1.07,1.27
4,Beta Q5,0.89,0.78,1.13,1.12,1.29


,Unnamed: 0,Cyber Q1,Cyber Q2,Cyber Q3,Cyber Q4,Cyber Q5
0,BM Q1,1.08,0.99,1.12,1.13,1.24
1,BM Q2,0.85,0.78,1.18,1.19,1.37
2,BM Q3,0.90,0.93,1.15,1.09,1.34
3,BM Q4,0.86,0.89,1.15,1.12,1.34
4,BM Q5,0.85,0.90,1.13,1.14,1.34


,Unnamed: 0,Cyber Q1,Cyber Q2,Cyber Q3,Cyber Q4,Cyber Q5
0,Size Q1,0.91,0.84,1.13,1.15,1.34
1,Size Q2,0.92,0.83,1.14,1.20,1.31
2,Size Q3,0.90,0.87,1.17,1.13,1.32
3,Size Q4,0.86,0.86,1.20,1.12,1.34
4,Size Q5,1.13,1.15,1.19,1.19,1.38


In [116]:
concatenated_df_horizontal = pd.concat([a1, a2, a3], axis=1)

In [117]:
concatenated_df_horizontal

,Unnamed: 0,Cyber Q1,Cyber Q2,Cyber Q3,Cyber Q4,Cyber Q5,Unnamed: 0,Cyber Q1,Cyber Q2,Cyber Q3,Cyber Q4,Cyber Q5,Unnamed: 0,Cyber Q1,Cyber Q2,Cyber Q3,Cyber Q4,Cyber Q5
0,Beta Q1,1.10,0.91,1.28,1.21,1.41,BM Q1,1.08,0.99,1.12,1.13,1.24,Size Q1,0.91,0.84,1.13,1.15,1.34
1,Beta Q2,0.93,0.87,1.20,1.13,1.35,BM Q2,0.85,0.78,1.18,1.19,1.37,Size Q2,0.92,0.83,1.14,1.20,1.31
2,Beta Q3,0.92,0.90,1.13,1.11,1.29,BM Q3,0.90,0.93,1.15,1.09,1.34,Size Q3,0.90,0.87,1.17,1.13,1.32
3,Beta Q4,0.90,0.77,1.20,1.07,1.27,BM Q4,0.86,0.89,1.15,1.12,1.34,Size Q4,0.86,0.86,1.20,1.12,1.34
4,Beta Q5,0.89,0.78,1.13,1.12,1.29,BM Q5,0.85,0.90,1.13,1.14,1.34,Size Q5,1.13,1.15,1.19,1.19,1.38


# Fama McBeth

In [50]:
path="results_pipeline/super/Preparation_and_Reconaissance"


In [51]:
def reader_fama_mcbeth(path):
    df = pd.read_csv(path)
    df.rename(columns={'Unnamed: 0': ' '}, inplace=True)
    df.set_index(' ', inplace=True)
    df=df.round(3)
    df = df.fillna("")
    df=df.astype(str)

    df["Average"]=df["Average"]+df["significance"]
    df.drop(columns=["significance","P-Value"],inplace=True)

    # Create a new DataFrame to hold the result
    new_data = []
    new_index = []
    
    for idx, row in df.iterrows():
        new_data.append([row['Average']])
        new_index.append(idx)
        if row['T-Statistic']:
            new_data.append([row['T-Statistic']])
            new_index.append(idx + '_T-Statistic')
    
    # Convert the new data into a DataFrame
    df = pd.DataFrame(new_data, index=new_index, columns=['Value'])


    return df

In [52]:
df1=reader_fama_mcbeth(path+"/model_1_FamaMcBeth_regression.csv").rename(columns={'Value': 'M.1'})
df2=reader_fama_mcbeth(path+"/model_2_FamaMcBeth_regression.csv").rename(columns={'Value': 'M.2'})
df3=reader_fama_mcbeth(path+"/model_3_FamaMcBeth_regression.csv").rename(columns={'Value': 'M.3'})
df4=reader_fama_mcbeth(path+"/model_4_FamaMcBeth_regression.csv").rename(columns={'Value': 'M.4'})
df5=reader_fama_mcbeth(path+"/model_5_FamaMcBeth_regression.csv").rename(columns={'Value': 'M.5'})



In [53]:
df = pd.merge(df5, df4, left_index=True, right_index=True, how='outer')
df = pd.merge(df, df3, left_index=True, right_index=True, how='outer')
df = pd.merge(df, df2, left_index=True, right_index=True, how='outer')
df = pd.merge(df, df1, left_index=True, right_index=True, how='outer')
df.fillna("", inplace=True)
df = df[['M.1','M.2','M.3','M.4','M.5']].astype(str)


In [54]:
df=df.reindex([
    "Mkt-RF",
    "Mkt-RF_T-Statistic",
    "cyber",
    "cyber_T-Statistic",
    "HML",
    "HML_T-Statistic",
    "SMB",
    "SMB_T-Statistic",
    "UMD",
    "UMD_T-Statistic",
    "CMA",
    "CMA_T-Statistic",
    "intercept",
    "intercept_T-Statistic",
    "R2",
    "MAPE"])

In [55]:
# Custom function to transform numbers
def transform_number(x):
    if x=="":
        return x
    return "["+x+"]"

# Apply the transformation only to rows with 'T-Statistics' in the index
df.loc[df.index.str.contains('T-Statistic')] = df.loc[df.index.str.contains('T-Statistic')].map(transform_number)


In [56]:
df = df.rename(index={
    "Mkt-RF_T-Statistic":"",
    "cyber_T-Statistic":"",
    "HML_T-Statistic":"",
    "SMB_T-Statistic":"",
    "UMD_T-Statistic":"",
    "CMA_T-Statistic":"",
    "intercept_T-Statistic":"",
    "intercept":"Constant",
    "Mkt-RF":"Market",
    "R2":"\overline{R2_{adj}}"
    })

<>:11: SyntaxWarning: invalid escape sequence '\o'
<>:11: SyntaxWarning: invalid escape sequence '\o'
C:\Users\natha\AppData\Local\Temp\ipykernel_13304\42343808.py:11: SyntaxWarning: invalid escape sequence '\o'
  "R2":"\overline{R2_{adj}}"


In [57]:
df

,M.1,M.2,M.3,M.4,M.5
Market,0.009*,,0.006,0.011**,0.01**
,[1.951],,[1.531],[2.541],[2.055]
cyber,,0.047*,0.046*,0.049**,0.052**
,,[1.848],[1.888],[2.262],[2.299]
HML,,,,0.003,0.004
,,,,[1.053],[1.52]
SMB,,,,0.001,0.001
,,,,[0.233],[0.418]
UMD,,,,0.001,
,,,,[0.168],


# GRS

In [168]:
path="results_pipeline/super/Preparation_and_Reconaissance"

In [169]:
df1 = pd.read_csv(path+"/GRS_cyber_involved.csv")
df1.rename(columns={'Unnamed: 0': ' '}, inplace=True)
df1.set_index(' ', inplace=True)
df1=df1.round(3)




df2 = pd.read_csv(path+"/GRS_no_cyber_involved.csv")
df2.rename(columns={'Unnamed: 0': ' '}, inplace=True)
df2.set_index(' ', inplace=True)
df2=df2.round(3)

In [170]:
df1

,GRS,p-value,R2
,,,
cyber,1.216,0.248,0.879
mkt,1.546,0.073,0.807
size,0.807,0.703,0.877
book,0.990,0.477,0.896


In [171]:
df=pd.concat([df2, df1])
df=df.iloc[[0,4,1,5,2,6,3,7]]

In [172]:
df1=df[0:4].reset_index()
df2=df[4:].reset_index()

In [173]:
df= pd.merge(df1, df2, left_index=True, right_index=True)

In [174]:
df.drop(columns=[" _x"," _y"],inplace=True)

In [175]:
df

,GRS_x,p-value_x,R2_x,GRS_y,p-value_y,R2_y
0,1.516,0.083,0.861,0.737,0.783,0.876
1,1.216,0.248,0.879,0.807,0.703,0.877
2,1.541,0.075,0.793,1.240,0.229,0.891
3,1.546,0.073,0.807,0.990,0.477,0.896
